In [ ]:
# 기사 목록

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup
import urllib.parse
from datetime import datetime

# 크롬 드라이버 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 필요시 활성화
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--blink-settings=imagesEnabled=false")  # 이미지 로드 비활성화
chrome_options.add_argument("--disable-extensions")  # 확장 프로그램 비활성화
chrome_options.add_argument("--disable-gpu")  # GPU 가속 비활성화

# 키워드 목록과 날짜 범위 설정
keywords = [
    "검색어",
]
start_date_str = "2024-06-01"
end_date_str = "2024-08-31"
seen_urls = set() 

# 크롬 드라이버 설정
driver = webdriver.Chrome(options=chrome_options)

def parse_date(date_str):
    """날짜 문자열을 datetime 객체로 변환."""
    try:
        return datetime.strptime(date_str, "%Y-%m-%d")
    except ValueError:
        print(f"Invalid date format: {date_str}")
        return None

def fetch_search_results(keyword, start_date, end_date):
    base_url = "https://sports.chosun.com/search/?s="
    results = []
    page = 1

    while True:
        print(f"{keyword} {page} 수집 중...")
        search_url = base_url +  urllib.parse.quote(keyword) + f"&page={page}"
        
        driver.get(search_url)

        try:
            # 페이지가 로드될 때까지 대기
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.serchList")))
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            articles = soup.find_all('div', class_='single-blog-post')

            print(f"찾은 기사 수: {len(articles)}")  # 로그 추가

            if not articles:
                print(f"{keyword} - {page} 페이지에서 기사를 찾지 못했습니다. 수집을 종료합니다.")
                break  # 기사가 없으면 루프 종료

        except Exception as e:
            print(f"{keyword} - {page} 페이지 로딩 중 오류 발생: {e}")
            break  # 페이지 로드 오류 시 루프 종료

        # 기사 정보 추출
        for article in articles:
            title_tag = article.select_one('div.post-data div.post-meta a.post-title')
            date_tag = article.select_one('div.post-data div.post-meta p.post-day')
            
            if title_tag and date_tag:
                title = title_tag.get_text(strip=True)
                url = title_tag['href']
                date_str = date_tag.get_text(strip=True).split()[0]

                # 중복 URL 체크
                if url in seen_urls:
                    continue  # 중복된 URL은 무시
                seen_urls.add(url)

                # 날짜 파싱
                article_date = parse_date(date_str)
                if article_date is None:
                    continue  # 잘못된 날짜 형식은 스킵

                # 시작 날짜 이전 기사면 종료
                if article_date < start_date:
                    print(f"기사가 시작 날짜 이전입니다. 스크래핑을 중단합니다: {date_str}")
                    return results  # 루프 종료

                # 종료 날짜 이후의 기사는 무시
                if article_date > end_date:
                    continue  # 범위 밖의 기사는 스킵

                # 날짜 범위 내 기사만 추가
                if start_date <= article_date <= end_date:
                    results.append({
                        "media": '스포츠조선',
                        "title": title,
                        "date": date_str,
                        "section": '',
                        "article_type": '',
                        'writer': '',
                        "page": '',
                        "url": url,
                        'content': ''
                    })

        print(f"{keyword} - {page} 페이지를 처리")

        # 다음 페이지로 이동
        page += 1
                          
    return results

def save_to_excel(data, filename):
    df = pd.DataFrame(data)
    df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
    df.to_excel(filename, index=False)
# 날짜 문자열을 datetime 객체로 변환
start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
end_date = datetime.strptime(end_date_str, "%Y-%m-%d")

# 모든 키워드에 대해 검색 수행
all_results = []
for keyword in keywords:
    results = fetch_search_results(keyword, start_date, end_date)
    all_results.extend(results)

# 결과를 엑셀 파일로 저장
save_to_excel(all_results, f'3스포츠조선 {start_date_str}~{end_date_str}.xlsx')

# 드라이버 종료
driver.quit()


In [ ]:
# 본문 추출
# 서브타이틀 x

import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# Selenium 헤드리스 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# 1. 출처와 파일명 변수 설정
file_name = '파일명'  # 파일명 설정
base_path = rf'경로'  # 경로 설정

# 2. 임시 파일 경로와 최종 파일 경로 생성
temp_output_path = os.path.join(base_path, f'{file_name}.xlsx')
# 임시 파일 에러를 대비해 본 파일에 덮어쓴다.
output_path = os.path.join(base_path, f'{file_name}_본문.xlsx')  # 최종 파일 경로
error_log_path = os.path.join(base_path, f'{file_name}_error_log.txt')  # 에러 로그 파일 경로

# 3. 파일 경로 생성 및 엑셀 파일 불러오기
input_file = os.path.join(base_path, f'{file_name}.xlsx')
df = pd.read_excel(input_file, engine='openpyxl')

# ChromeDriver 설정 없이 크롬 브라우저 실행
driver = webdriver.Chrome(options=chrome_options)

# URL에서 본문을 스크랩하는 함수 정의
def get_content_from_url(url):
    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        article_body = soup.find('div', class_='news_text')

        if article_body:
            # 'caption' 클래스를 가진 모든 태그를 제거합니다.
            for caption in article_body.find_all('table', align='center'):
                caption.decompose()
            content = article_body.get_text(separator='\n', strip=True)
        else:
            content = ""
        return content
    except Exception as e:
        print(f"Error fetching URL {url}: {e}")
        return None

# 진행 중인 기사 스크래핑 상태를 트래킹할 리스트와 로그 파일
error_log = []
total_articles = len(df)

# 각 행의 URL에서 본문 내용을 스크랩하여 content 컬럼에 저장합니다.
for index, row in df.iterrows():
    print(f"Processing {index + 1}/{total_articles}: {row['url']}")
    content = get_content_from_url(row['url'])
    if content is None:
        error_log.append(row['url'])
    df.at[index, 'content'] = content

        # 100개마다 파일 저장
    if (index + 1) % 100 == 0:
        df.to_excel(temp_output_path, index=False)
        print(f"임시 파일을 저장했습니다: {temp_output_path}")

# 웹드라이버 종료
driver.quit()

# 에러 로그 파일 저장
if error_log:
    with open(error_log_path, 'w') as f:
        for url in error_log:
            f.write(f"{url}\n")
    print(f"에러 로그가 저장되었습니다: {error_log_path}")

# 변경된 데이터를 파일로 저장합니다.
df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df = df[~df['content'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df.to_excel(output_path, index=False)

print("본문 내용을 성공적으로 스크랩하여 저장했습니다.")      
